In [1]:
pip install pyodbc pandas


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyodbc
import pandas as pd

# Example connection string (update with your server and credentials)
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-R0KQMS9S\\MSSQLSERVER02;'
    'DATABASE=Bank_Review;'
    'Trusted_Connection=yes'
)
cursor = conn.cursor()



In [10]:
import pandas as pd

# Load cleaned data
df = pd.read_csv("C:/Users/hp/OneDrive - menbimarket/Desktop/Customer-Experience-Analytics-for-Fintech-Apps/data/task2_sentiment_analysis.csv")

# Check the structure
print(df.head())




                                              review  rating        date  \
0       the app is proactive and a good connections.       5  2025-06-05   
1    I cannot send to cbebirr app. through this app.       3  2025-06-05   
2                                               good       4  2025-06-05   
3                                     not functional       1  2025-06-04   
4  everytime you uninstall the app you have to re...       1  2025-06-04   

                          bank       source  \
0  Commercial Bank of Ethiopia  Google Play   
1  Commercial Bank of Ethiopia  Google Play   
2  Commercial Bank of Ethiopia  Google Play   
3  Commercial Bank of Ethiopia  Google Play   
4  Commercial Bank of Ethiopia  Google Play   

                                        cleaned_text sentiment  \
0                      app proactive good connection  POSITIVE   
1                               send cbebirr app app  NEGATIVE   
2                                               good  POSITIVE

In [22]:
# Convert to float and fill NaN with a default (e.g., 0.0 or skip the row)
df['sentiment_score'] = pd.to_numeric(df['sentiment_score'], errors='coerce')  # Convert invalid to NaN
df = df.dropna(subset=['sentiment_score'])  # Optionally drop rows with invalid scores


In [23]:
df['sentiment_score'] = pd.to_numeric(df['sentiment_score'], errors='coerce').fillna(0.0)


In [28]:
# Step 1: Insert banks and build a mapping
bank_names = df['bank'].unique()
bank_id_map = {}

for bank in bank_names:
    cursor.execute("INSERT INTO banks (bank_name, app_name) VALUES (?, ?)", (bank, f"{bank} App"))
    conn.commit()
    cursor.execute("SELECT bank_id FROM banks WHERE bank_name = ?", (bank,))
    bank_id_map[bank] = cursor.fetchone()[0]



In [31]:
df_banks = pd.read_sql_query("SELECT * FROM banks", conn)
df_banks.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_21888\4008231484.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_banks = pd.read_sql_query("SELECT * FROM banks", conn)


,bank_id,bank_name,app_name,country
0,1,Commercial Bank of Ethiopia,Commercial Bank of Ethiopia App,Ethiopia
1,2,Bank of Abyssinia,Bank of Abyssinia App,Ethiopia
2,3,Dashen Bank,Dashen Bank App,Ethiopia


In [26]:
# Run the UPDATE query
cursor.execute("UPDATE banks SET country = 'Ethiopia'")
conn.commit()

print("Country column updated successfully!")


Country column updated successfully!


In [40]:
df_banks.head()

,bank_id,bank_name,app_name,country
0,1,Commercial Bank of Ethiopia,Commercial Bank of Ethiopia App,Ethiopia
1,2,Bank of Abyssinia,Bank of Abyssinia App,Ethiopia
2,3,Dashen Bank,Dashen Bank App,Ethiopia


In [39]:
df['sentiment_score']=df['sentiment_score'].apply(pd.to_numeric, errors='coerce')




In [41]:
df['sentiment_score']

0       0.999832
1       0.979366
2       0.999816
3       0.999333
4       0.994766
          ...   
1420    0.999570
1421    0.999627
1422    0.999860
1423    0.999770
1424    0.999778
Name: sentiment_score, Length: 1425, dtype: float64

In [44]:
# Assuming you have a connection object 'conn' and a cursor 'cursor'
cursor = conn.cursor()

# Query bank id and bank name from banks table
cursor.execute("SELECT bank_id, bank_name FROM banks")
banks = cursor.fetchall()

# Create a dictionary: key = bank_name, value = bank_id
bank_id_map = {bank_name.strip(): bank_id for bank_id, bank_name in banks}

print(bank_id_map)  # to verify



{'Commercial Bank of Ethiopia': 1, 'Bank of Abyssinia': 2, 'Dashen Bank': 3}


In [52]:
# --- Step 3: Get bank_id mapping from the database ---
cursor.execute("SELECT bank_id, bank_name FROM banks")
bank_map = {row.bank_name: row.bank_id for row in cursor.fetchall()}
bank_map

{'Commercial Bank of Ethiopia': 1, 'Bank of Abyssinia': 2, 'Dashen Bank': 3}

In [54]:
# --- Step 4: Map bank_name → bank_id in the DataFrame ---
df['bank_id'] = df['bank'].map(bank_map)


In [55]:
df

,review,rating,date,bank,source,cleaned_text,sentiment,sentiment_score,sentiment_label,themes,bank_id
0,the app is proactive and a good connections.,5,2025-06-05,Commercial Bank of Ethiopia,Google Play,app proactive good connection,POSITIVE,0.999832,positive,['Other'],1
1,I cannot send to cbebirr app. through this app.,3,2025-06-05,Commercial Bank of Ethiopia,Google Play,send cbebirr app app,NEGATIVE,0.979366,negative,['Other'],1
2,good,4,2025-06-05,Commercial Bank of Ethiopia,Google Play,good,POSITIVE,0.999816,positive,['Other'],1
3,not functional,1,2025-06-04,Commercial Bank of Ethiopia,Google Play,functional,POSITIVE,0.999333,positive,['Other'],1
4,everytime you uninstall the app you have to re...,1,2025-06-04,Commercial Bank of Ethiopia,Google Play,everytime uninstall app reach physically oldy ...,NEGATIVE,0.994766,negative,['Other'],1
...,...,...,...,...,...,...,...,...,...,...,...
1420,Best,5,2025-01-13,Dashen Bank,Google Play,well,POSITIVE,0.999570,positive,['Other'],3
1421,"Waw Great and innovated,user friendly, always ...",5,2025-01-13,Dashen Bank,Google Play,waw great innovateduser friendly always one st...,POSITIVE,0.999627,positive,['Other'],3
1422,It's Best waww 🙏,5,2025-01-12,Dashen Bank,Google Play,good waww,POSITIVE,0.999860,positive,['Other'],3
1423,Always one step ahead,5,2025-01-12,Dashen Bank,Google Play,always one step ahead,POSITIVE,0.999770,positive,['Other'],3


In [56]:
if df['bank_id'].isnull().any():
    print("⚠️ Unmatched bank names:")
    print(df[df['bank_id'].isnull()]['bank_name'].unique())

# Drop unmatched rows
df = df.dropna(subset=['bank_id'])

In [60]:
# Check for non-numeric or missing values
print(df[['sentiment_score', 'rating']].dtypes)
print(df[df['sentiment_score'].isnull()])
print(df[df['rating'].isnull()])


sentiment_score    float64
rating               int64
dtype: object
Empty DataFrame
Columns: [review, rating, date, bank, source, cleaned_text, sentiment, sentiment_score, sentiment_label, themes, bank_id]
Index: []
Empty DataFrame
Columns: [review, rating, date, bank, source, cleaned_text, sentiment, sentiment_score, sentiment_label, themes, bank_id]
Index: []


In [67]:
new_df=df.drop(columns=['review','source','sentiment','themes'])
new_df

,rating,date,bank,cleaned_text,sentiment_score,sentiment_label,bank_id
0,5,2025-06-05,Commercial Bank of Ethiopia,app proactive good connection,0.999832,positive,1
1,3,2025-06-05,Commercial Bank of Ethiopia,send cbebirr app app,0.979366,negative,1
2,4,2025-06-05,Commercial Bank of Ethiopia,good,0.999816,positive,1
3,1,2025-06-04,Commercial Bank of Ethiopia,functional,0.999333,positive,1
4,1,2025-06-04,Commercial Bank of Ethiopia,everytime uninstall app reach physically oldy ...,0.994766,negative,1
...,...,...,...,...,...,...,...
1420,5,2025-01-13,Dashen Bank,well,0.999570,positive,3
1421,5,2025-01-13,Dashen Bank,waw great innovateduser friendly always one st...,0.999627,positive,3
1422,5,2025-01-12,Dashen Bank,good waww,0.999860,positive,3
1423,5,2025-01-12,Dashen Bank,always one step ahead,0.999770,positive,3


In [68]:
new_df.to_csv('cleaned_reviews_final.csv', index=False)

In [61]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])
pd.to_datetime(row['date'])


Timestamp('2025-06-04 00:00:00')

In [63]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])  # Drop rows where date is invalid


In [59]:
# --- Step 5: Prepare for INSERT ---
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO reviews (
            bank_id, review_text, sentiment_label, sentiment_score, 
            review_date, rating
        )
        VALUES (?, ?, ?, ?, ?, ?)
    """, (
        int(row['bank_id']),
        row['cleaned_text'],
        row['sentiment_label'],
        float(row['sentiment_score']),
        pd.to_datetime(row['date']),
        int(row['rating'])
        
    ))
                

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 5 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 5 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')

In [51]:
print (row['cleaned_text'])

nan
